In [ ]:
#| default_exp core

# Core

Useful functions and utilities for LibreGrid.

In [10]:
#| export
import httpx
import asyncio
import os
import requests
import random
import sqlite3
import aiosqlite
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta

In [2]:
# Fetching environment variables
load_dotenv()
solar_key = os.environ['API_SOLAR_KEY']

In [ ]:
#| export
def get_smart_meter_data():
    "Python function to get smart meter data from Octopus Home Mini API"
    OE_API_KEY = os.getenv("OE_API_KEY")
    # TODO

In [3]:
#| export
def get_solar_building_insights(solar_key, latitude, longitude):
    url = 'https://solar.googleapis.com/v1/buildingInsights:findClosest'
    params = {
    'location.latitude': latitude,
    'location.longitude': longitude,
    'requiredQuality': 'LOW',
    'key':solar_key
    }
    response = requests.get(url, params=params)
    return response.json()

In [11]:
# Load the JSON file
with open('solar_api_response.json', 'r') as file:
    data = json.load(file)

In [38]:
def return_energy(building_insights):
    third = round(building_insights['solarPotential']['maxArrayPanelsCount']/3)
    
    energy_dc = (item for item in building_insights['solarPotential']['solarPanelConfigs'] if (item["panelsCount"] == third or item["panelsCount"] == third+1 or item["panelsCount"] == third-1))
    
    return list(energy_dc)[0]['yearlyEnergyDcKwh']

In [39]:
print(return_energy(data))

224882.6


In [4]:
## test = get_solar_building_insights(solar_key=solar_key, latitude=37.4449739, longitude=-122.13914659999998)

In [3]:
def fetch_meter_usage(MPAN_import,serial_number):
    api_key = os.environ['phil_key']
    url = f"https://api.octopus.energy/v1/electricity-meter-points/{MPAN_import}/meters/{serial_number}/consumption/"
    params = {
        "period_from":"2024-01-01 00:00:00",
        "period_to":"2024-07-30 00:00:00",
        "page_size":"20"
    }
    response = requests.get(url,auth=(api_key,''), params = params)
    return response.json()
    

In [4]:
test = fetch_meter_usage(os.environ['MPAN_import'],os.environ['serial_number'])

In [55]:
#| export
def fetch_latest_meter_data():
    "Function to get last hour of meter consumption data" 
    api_key = os.environ['phil_key']
    mpan = os.environ['MPAN_import']
    serial_num = os.environ['serial_number']

    end_time = datetime.now()
    start_time = end_time - timedelta(hours=10)
    url = f"https://api.octopus.energy/v1/electricity-meter-points/{mpan}/meters/{serial_num}/consumption/"
    params = {
        #"period_from":start_time.isoformat(),
        #"period_to":end_time.isoformat(),
        #'group_by':'hour',
        "page_size":"2"
    }
    
    response = requests.get(url, auth=(api_key, ''), params=params)
    if response.status_code == 200:
        data = response.json()
        results = data['results'] if data['results'] else None
        if results:
            for result in results:
                result['mpan_import'] = mpan
                result['serial_number'] = serial_num
        return results
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None
    

In [6]:
#| export
def round_now_to_last_half_hour():
    "Helper function to round current time to last half hour"
    dt = datetime.now()
    rounded = dt.replace(second=0, microsecond=0)
    if rounded.minute >= 30:
        rounded = rounded.replace(minute=30)
    else:
        rounded = rounded.replace(minute=0)
    return rounded

In [96]:
#| export
def dummy_meter(page_size=2):
    "Meter data simulator that just generates random consumption values at every full half hour, at a given interval"
    responses = []
    for item in range(0,page_size): 
        start_time = round_now_to_last_half_hour() - timedelta(minutes=30*(item+1))
        end_time = round_now_to_last_half_hour() - timedelta(minutes=30*(item))
        entry = {
            'mpan_import': 'dummy',
            'serial_number': f'dummy:{int(random.uniform(1,100))}',
            'consumption': round(random.uniform(0, 0.3), 3),
            'interval_start': start_time.isoformat(),
            'interval_end': end_time.isoformat()
        }
        responses.append(entry)
    return responses
        
    

### Section setting up a database for the meter data and inserting it

In [111]:
# Set up a database
connection = sqlite3.connect('libregrid.db')

In [112]:
# Set up my table for meter data
cursor = connection.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS meter_data (
mpan_import TEXT,
serial_number TEXT,
interval_start TEXT,
interval_end TEXT,
consumption REAL,
PRIMARY KEY (mpan_import, serial_number, interval_start)
)
''')
connection.commit()

In [81]:
# Function to insert data 
async def insert_meter_data(consumption_list):
    async with aiosqlite.connect('libregrid.db') as db:
        await db.executemany('''
        INSERT OR REPLACE INTO meter_data (mpan_import, serial_number, interval_start, interval_end, consumption)
        VALUES (:mpan_import, :serial_number, :interval_start, :interval_end, :consumption)
        ''', consumption_list)
        await db.commit()

In [74]:
# Function to get data
async def get_meter_data():
    async with aiosqlite.connect('libregrid.db') as db:
        async with db.execute('SELECT * FROM meter_data') as cursor:
            rows = await cursor.fetchall()
            columns = [description[0] for description in cursor.description]
            return [dict(zip(columns, row)) for row in rows]

In [95]:
result = dummy_meter()
result

[{'mpan_import': 'dummy',
  'serial_number': 'dummy:2',
  'consumption': 0.031,
  'interval_start': '2024-08-20T16:30:00',
  'interval_end': '2024-08-20T17:00:00'},
 {'mpan_import': 'dummy',
  'serial_number': 'dummy:7',
  'consumption': 0.29,
  'interval_start': '2024-08-20T16:00:00',
  'interval_end': '2024-08-20T16:30:00'}]

In [ ]:
await insert_meter_data(result)

In [100]:
result_real = await fetch_latest_meter_data()
result_real


[{'consumption': 0.144,
  'interval_start': '2024-08-20T00:30:00+01:00',
  'interval_end': '2024-08-20T01:00:00+01:00',
  'mpan_import': '1413985791007',
  'serial_number': '21J0016813'},
 {'consumption': 0.143,
  'interval_start': '2024-08-20T00:00:00+01:00',
  'interval_end': '2024-08-20T00:30:00+01:00',
  'mpan_import': '1413985791007',
  'serial_number': '21J0016813'}]

In [113]:
await insert_meter_data(result_real)

In [114]:
await get_meter_data()

[{'mpan_import': '1413985791007',
  'serial_number': '21J0016813',
  'interval_start': '2024-08-20T00:30:00+01:00',
  'interval_end': '2024-08-20T01:00:00+01:00',
  'consumption': 0.144},
 {'mpan_import': '1413985791007',
  'serial_number': '21J0016813',
  'interval_start': '2024-08-20T00:00:00+01:00',
  'interval_end': '2024-08-20T00:30:00+01:00',
  'consumption': 0.143}]

In [115]:
connection.close()